In [8]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import os
from PIL import Image

from shutil import copyfile
import base64
from io import BytesIO
from IPython.display import HTML
import pickle
import imutils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Activation
from keras.layers import Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import load_model

from mtcnn.mtcnn import MTCNN
# print version

import cv2
import copy

from tqdm import tqdm
import glob

In [9]:
augmented_images_df = pd.read_pickle('aug_df_2.pkl')

In [10]:
augmented_images_df

,neutralised_image,neutralised_original_image,crop_original_image,original_image,original_emotion,name,all_images,all_original_images
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[14, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 42, 39, 34, 32, 23, 8, 1, 1, 0, 0, 0, 0,...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 42, 39, 34, 32, 23, 8, 1, 1, 0, 0, 0, 0,..."
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[14, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 42, 39, 34, 32, 23, 8, 1, 1, 0, 0, 0, 0,...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[14, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[40, 33, 28, 24, 10, 1, 0, 0, 0, 0, 0, 0, 0, ...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[40, 33, 28, 24, 10, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[40, 33, 28, 24, 10, 1, 0, 0, 0, 0, 0, 0, 0, ...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[19, 7, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[19, 7, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[19, 7, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[34, 26, 12, 4, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[40, 41, 41, 37, 33, 34, 35, 27, 10, 1, 0, 0,...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[40, 41, 41, 37, 33, 34, 35, 27, 10, 1, 0, 0,..."
7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[34, 26, 12, 4, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[40, 41, 41, 37, 33, 34, 35, 27, 10, 1, 0, 0,...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[34, 26, 12, 4, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,..."
8,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 43, 38, 32, 26, 18, 5, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 41, 38, 35, 32, 25, 9, 0, 0, 0, 0, 0, 0,...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 41, 38, 35, 32, 25, 9, 0, 0, 0, 0, 0, 0,..."
9,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 43, 38, 32, 26, 18, 5, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 41, 38, 35, 32, 25, 9, 0, 0, 0, 0, 0, 0,...",disgust,111,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[44, 43, 38, 32, 26, 18, 5, 0, 0, 0, 0, 0, 0,..."


In [11]:
augmented_images_df = augmented_images_df.sample(frac=1)

In [12]:
augmented_images_df = augmented_images_df.iloc[:]

In [13]:
X_rec_origin, y_rec_origin = augmented_images_df[['all_images', 'crop_original_image', 'neutralised_image']], augmented_images_df['name']

X_rec_origin_train, X_rec_origin_test, y_rec_origin_train, y_rec_origin_test = train_test_split(X_rec_origin, y_rec_origin, test_size=0.33, random_state=42)
X_rec_origin_train.reset_index(drop=True, inplace=True), X_rec_origin_test.reset_index(drop=True, inplace=True)
y_rec_origin_train.reset_index(drop=True, inplace=True), y_rec_origin_test.reset_index(drop=True, inplace=True)
X_rec_origin_train.shape, X_rec_origin_test.shape, y_rec_origin_train.shape, y_rec_origin_test.shape

((7873, 3), (3879, 3), (7873,), (3879,))